In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

/content/drive/MyDrive/Colab Notebooks/blackjack_simulator.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive//MyDrive/blackjack/blackjack_simulator.csv', chunksize=50000)

In [ ]:
data = next(data)

In [ ]:
data.head()

,shoe_id,cards_remaining,dealer_up,initial_hand,dealer_final,dealer_final_value,player_final,player_final_value,actions_taken,run_count,true_count,win
0,0,416,10,"[10, 11]","[10, 4, 10]",24,"[[10, 11]]",['BJ'],[['S']],1,0,1.5
1,0,411,10,"[5, 5]","[10, 8]",18,"[[5, 5, 11]]",[21],"[['H', 'S']]",-2,0,1.0
2,0,406,6,"[3, 10]","[6, 6, 10]",22,"[[3, 10]]",[13],[['S']],-2,0,1.0
3,0,401,10,"[5, 9]","[10, 8]",18,"[[5, 9, 11, 3]]",[18],"[['H', 'H', 'S']]",-1,0,0.0
4,0,395,8,"[6, 10]","[8, 2, 10]",20,"[[6, 10, 10]]",[26],[['H']],-1,0,-1.0


In [ ]:
##data cleaning
#Make a new column for every new sequence, with df.explode
#delete all the rows that have multiple lists
#Convert all strings into lists


In [ ]:
import ast
# cleaning and preprocessing data
data['player_final_value'] = data['player_final_value'].apply(lambda x: ast.literal_eval(x))
data['actions_taken'] = data['actions_taken'].apply(lambda x: ast.literal_eval(x))
data['initial_hand'] = data['initial_hand'].apply(lambda x: ast.literal_eval(x))
data['dealer_final'] = data['dealer_final'].apply(lambda x: ast.literal_eval(x))
data['player_final'] = data['player_final'].apply(lambda x: ast.literal_eval(x))
data = data.loc[data.actions_taken.map(len)<2]
data = data.loc[data.player_final.map(len)<2]
data['actions_taken'] = data['actions_taken'].apply(lambda x: [x[0] , ['P'] + x[1]]  if len(x)==2 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2]]  if len(x)==3 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2], ['P'] + x[3]]  if len(x)==4 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2], ['P'] + x[3] , ['P'] + x[4]]  if len(x)==5 else x)
data = data.explode(['player_final','player_final_value','actions_taken'])

def cards_splitter(data,cards,text):
    stopsign = cards.apply(lambda x: len(x))
    for i in range(1,max(stopsign)):
        data[f'{text}{i}'] = cards.apply(lambda x: None if len(x)<i else x[i-1])
cards_splitter(data,data['player_final'],'player_card_')
cards_splitter(data,data['dealer_final'],'dealer_card_')
cards_splitter(data,data['actions_taken'],'action_taken_')
data['is_blackjack_dealer'] = data['dealer_final_value'].where(data['dealer_final_value'] == 'BJ').fillna(0).replace('BJ',1)
data['is_blackjack_player'] = data['player_final_value'].where(data['player_final_value'] == 'BJ').fillna(0).replace('BJ',1)
data['dealer_final_value'] = data['dealer_final_value'].replace('BJ',21)
data['player_final_value'] = data['player_final_value'].replace('BJ',21)
data['dealer_final_value'] = data['dealer_final_value'].astype('int64')
data['player_final_value'] = data['player_final_value'].astype('int64')
data.loc[(data['dealer_final_value'] < data['player_final_value']) & (data['player_final_value'] <= 21) , 'win'] = 1
data.loc[data['dealer_final_value'] >  21 , 'win'] = 1
data.loc[data['dealer_final_value'] == data['player_final_value'], 'win'] = 0
data.loc[(data['dealer_final_value'] > data['player_final_value']) & (data['dealer_final_value'] <= 21) , 'win'] = -1
data.loc[data['player_final_value'] > 21 , 'win'] = -1
data['win'] = data['win'].astype('int64')
data.loc[(data['dealer_final_value'] < data['player_final_value']) & (data['player_final_value'] <= 21) , 'win'] = 1
data.loc[data['dealer_final_value'] >  21 , 'win'] = 1
data.loc[data['dealer_final_value'] == data['player_final_value'], 'win'] = 0
data.loc[(data['dealer_final_value'] > data['player_final_value']) & (data['dealer_final_value'] <= 21) , 'win'] = -1
data.loc[data['player_final_value'] > 21 , 'win'] = -1
data['win'] = data['win'].astype('int64')
data.drop(['dealer_up','initial_hand','dealer_final','dealer_final_value','player_final','player_final_value','actions_taken'],axis=1,inplace=True)  # Duplicate dat
data.drop(['shoe_id'],axis=1,inplace=True) # Not relevan
data[['action_taken_1','action_taken_2','action_taken_3','action_taken_4','action_taken_5']] = data[['action_taken_1','action_taken_2','action_taken_3','action_taken_4','action_taken_5']].replace({ 'S' : 1, 'H' : 2 , 'D' : 3 , 'P' : 4 , 'R' : 5 , 'N' : 6})

data = data.fillna(0)

print(data.info())

if "action_taken_6" in data.columns:
  data = data.drop(columns=['action_taken_6'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48752 entries, 0 to 49999
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cards_remaining      48752 non-null  int64  
 1   run_count            48752 non-null  int64  
 2   true_count           48752 non-null  int64  
 3   win                  48752 non-null  int64  
 4   player_card_1        48752 non-null  int64  
 5   player_card_2        48752 non-null  int64  
 6   player_card_3        48752 non-null  float64
 7   player_card_4        48752 non-null  float64
 8   player_card_5        48752 non-null  float64
 9   player_card_6        48752 non-null  float64
 10  player_card_7        48752 non-null  float64
 11  dealer_card_1        48752 non-null  int64  
 12  dealer_card_2        48752 non-null  int64  
 13  dealer_card_3        48752 non-null  float64
 14  dealer_card_4        48752 non-null  float64
 15  dealer_card_5        48752 non-null 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from tempfile import mkdtemp
from shutil import rmtree

from xgboost import XGBRegressor

from sklearn import set_config
set_config(display = 'diagram')

# Sklearn preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, VarianceThreshold, SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error,  accuracy_score, classification_report, confusion_matrix, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.svm import SVR
from xgboost import XGBClassifier

In [ ]:
data.head(20)

,cards_remaining,run_count,true_count,win,player_card_1,player_card_2,player_card_3,player_card_4,player_card_5,player_card_6,...,dealer_card_5,dealer_card_6,dealer_card_7,action_taken_1,action_taken_2,action_taken_3,action_taken_4,action_taken_5,is_blackjack_dealer,is_blackjack_player
0,416,1,0,1,10,11,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1
1,411,-2,0,1,5,5,11.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0,0
2,406,-2,0,1,3,10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
3,401,-1,0,0,5,9,11.0,3.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,0,0
4,395,-1,0,-1,6,10,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0,0
5,389,-2,0,-1,3,4,11.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0,0
7,374,2,0,-1,10,8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
8,369,1,0,1,8,10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
9,364,0,0,1,6,8,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0,0
10,357,3,0,1,11,10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1


In [ ]:
# we will use xgboost as our first model to predict the win.

In [ ]:
# features X
X = data.drop(columns=['win'])

# target y
y = data.win
y = y.map({-1:0, 0:1, 1:2})


In [ ]:
# Split data into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 42  # TEST = 30%
)

# Use the same function above for the validation set
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size = 0.5, random_state = 42  # TEST = 15%
)

In [ ]:
from scipy.stats import randint, uniform

In [ ]:
gbm_param_grid = {
    'n_estimators': [100,150],
    'max_depth': randint(2, 12),
    'colsample_bytree' : uniform(0.05, 0.5),
    'learning_rate':np.linspace(0.01,2,100),
    'alpha' : np.linspace(0.01,100,100),
    'min_child_weight' : randint(1,20),
    'grow_policy':['depthwise', 'lossguide'],
    'refresh_leaf':[0,1],
    'scale_pos_weight':randint(1,100),
    'lambda':np.linspace(0.01,2,100),
    'tree_method':["gpu_hist"],
    'objective':['reg:squarederror'],
    'colsample_bylevel':np.linspace(0.01,1,100),
    }

In [ ]:
# initialize the model
xgb = XGBClassifier()
# define gridsearch
random_search = RandomizedSearchCV(estimator=xgb, param_distributions=gbm_param_grid, cv=5, n_iter=100)

# fit the model
random_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = random_search.best_params_
best_model = random_search.best_estimator_

[14:58:31] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:31] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:32] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:32] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:32] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:33] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:33] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:33] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:34] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:34] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[14:58:34] WARNING: ../src/learner.cc:767: 
Parameters: { "s

In [ ]:
best_params

In [ ]:
best_model

XGBClassifier(alpha=10.11, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8200000000000001, colsample_bynode=None,
              colsample_bytree=0.3950657415474103, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy='depthwise',
              importance_type=None, interaction_constraints=None,
              lambda=0.9145454545454546, learning_rate=0.35171717171717176,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=9, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
import joblib

In [ ]:
for i, data in enumerate(pd.read_csv('/content/drive//MyDrive/blackjack/blackjack_simulator.csv', chunksize=100000)):
    # cleaning and preprocessing data
    data['player_final_value'] = data['player_final_value'].apply(lambda x: ast.literal_eval(x))
    data['actions_taken'] = data['actions_taken'].apply(lambda x: ast.literal_eval(x))
    data['initial_hand'] = data['initial_hand'].apply(lambda x: ast.literal_eval(x))
    data['dealer_final'] = data['dealer_final'].apply(lambda x: ast.literal_eval(x))
    data['player_final'] = data['player_final'].apply(lambda x: ast.literal_eval(x))
    data = data.loc[data.actions_taken.map(len)<2]
    data = data.loc[data.player_final.map(len)<2]
    data['actions_taken'] = data['actions_taken'].apply(lambda x: [x[0] , ['P'] + x[1]]  if len(x)==2 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2]]  if len(x)==3 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2], ['P'] + x[3]]  if len(x)==4 else x).apply(lambda x: [x[0] , ['P'] + x[1], ['P'] + x[2], ['P'] + x[3] , ['P'] + x[4]]  if len(x)==5 else x)
    data = data.explode(['player_final','player_final_value','actions_taken'])

    def cards_splitter(data,cards,text):
        stopsign = cards.apply(lambda x: len(x))
        for i in range(1,max(stopsign)):
            data[f'{text}{i}'] = cards.apply(lambda x: None if len(x)<i else x[i-1])

    cards_splitter(data,data['player_final'],'player_card_')
    cards_splitter(data,data['dealer_final'],'dealer_card_')
    cards_splitter(data,data['actions_taken'],'action_taken_')

    data['is_blackjack_dealer'] = data['dealer_final_value'].where(data['dealer_final_value'] == 'BJ').fillna(0).replace('BJ',1)
    data['is_blackjack_player'] = data['player_final_value'].where(data['player_final_value'] == 'BJ').fillna(0).replace('BJ',1)

    data['dealer_final_value'] = data['dealer_final_value'].replace('BJ',21)
    data['player_final_value'] = data['player_final_value'].replace('BJ',21)

    data['dealer_final_value'] = data['dealer_final_value'].astype('int64')
    data['player_final_value'] = data['player_final_value'].astype('int64')

    data.loc[(data['dealer_final_value'] < data['player_final_value']) & (data['player_final_value'] <= 21) , 'win'] = 1
    data.loc[data['dealer_final_value'] >  21 , 'win'] = 1
    data.loc[data['dealer_final_value'] == data['player_final_value'], 'win'] = 0
    data.loc[(data['dealer_final_value'] > data['player_final_value']) & (data['dealer_final_value'] <= 21) , 'win'] = -1
    data.loc[data['player_final_value'] > 21 , 'win'] = -1
    data['win'] = data['win'].astype('int64')
    data.loc[(data['dealer_final_value'] < data['player_final_value']) & (data['player_final_value'] <= 21) , 'win'] = 1
    data.loc[data['dealer_final_value'] >  21 , 'win'] = 1
    data.loc[data['dealer_final_value'] == data['player_final_value'], 'win'] = 0
    data.loc[(data['dealer_final_value'] > data['player_final_value']) & (data['dealer_final_value'] <= 21) , 'win'] = -1
    data.loc[data['player_final_value'] > 21 , 'win'] = -1
    data['win'] = data['win'].astype('int64')

    data.drop(['dealer_up','initial_hand','dealer_final','dealer_final_value','player_final','player_final_value','actions_taken'],axis=1,inplace=True)  # Duplicate data

    data.drop(['shoe_id'],axis=1,inplace=True) # Not relevant

    data[['action_taken_1','action_taken_2','action_taken_3','action_taken_4', 'action_taken_5']] = data[['action_taken_1','action_taken_2','action_taken_3','action_taken_4', 'action_taken_5']].replace({ 'S' : 1, 'H' : 2 , 'D' : 3 , 'P' : 4 , 'R' : 5 , 'N' : 6})


    data = data.fillna(0)

    print(data.info())


    if "action_taken_6" in data.columns:
      data = data.drop(columns=['action_taken_6'])
    if 'action_taken_7' in data.columns:
      data = data.drop(columns=['action_taken_7'])


    # features X
    X = data.drop(columns=['win'])

    # target y
    y = data.win
    y = y.map({-1:0, 0:1, 1:2})

    # Split data into train, test and validation sets
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42  # TEST = 30%
    )

    # Use the same function above for the validation set
    X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size = 0.5, random_state = 42  # TEST = 15%
    )


    xgboost_sav_new = best_model.fit(X_train, y_train)

    if i == 50:
      joblib.dump(xgboost_sav_new, 'xgb_model_blackjack.sav')
      break







<class 'pandas.core.frame.DataFrame'>
Int64Index: 97520 entries, 0 to 99999
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cards_remaining      97520 non-null  int64  
 1   run_count            97520 non-null  int64  
 2   true_count           97520 non-null  int64  
 3   win                  97520 non-null  int64  
 4   player_card_1        97520 non-null  int64  
 5   player_card_2        97520 non-null  int64  
 6   player_card_3        97520 non-null  float64
 7   player_card_4        97520 non-null  float64
 8   player_card_5        97520 non-null  float64
 9   player_card_6        97520 non-null  float64
 10  player_card_7        97520 non-null  float64
 11  dealer_card_1        97520 non-null  int64  
 12  dealer_card_2        97520 non-null  int64  
 13  dealer_card_3        97520 non-null  float64
 14  dealer_card_4        97520 non-null  float64
 15  dealer_card_5        97520 non-null 

In [ ]:
# Make predictions on the test set
y_pred = xgboost_sav_new.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8977


In [ ]:
loaded_model_xgb = joblib.load('xgb_model_blackjack.sav')

In [ ]:
import joblib
joblib.dump(xbg_reg, "xgb_model_blackjack.sav")
xgb_reg = joblib.load("xgb_model_blackjack.sav")
preds = xgb_reg.predict(X_test)